In [2]:
import pandas as pd
import numpy as np
from gurobipy import *

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
from scipy.optimize import linprog

In [4]:
X_separable = np.array([[1],[2],[3],[4]])
y_separable = np.array([[-1],[-1],[1],[1]])
# If the X values are plotted on the X axis, there will be clear linear separation between label -1 and 1
# In this case the LP will give an optimal solution
# Indicates the MLE does not Exist

In [5]:
X_nonseparable = np.array([[1],[2],[3],[4]])
y_nonseparable = np.array([[-1],[1],[-1],[1]])
# If the X values are plotted on the X axis, there are no linear separation between label -1 and 1
# In this case the LP will say Infeasible solution
# Indicates the MLE Exists

# Separable

In [6]:
X = pd.DataFrame(X_separable)
X

,0
0,1
1,2
2,3
3,4


In [7]:
y= pd.DataFrame(y_separable)
y

,0
0,-1
1,-1
2,1
3,1


# Scipy Way (Separable) Internet Version

In [8]:
tmp = X.values
tmp = sc.fit_transform(tmp)


xx = np.array(y.values.reshape(-1,1) * tmp)
t = y.values

A_ub = np.append(xx, t.reshape(-1,1), 1)
b_ub = np.repeat(-1, A_ub.shape[0]).reshape(-1,1)

c_obj = np.repeat(1, A_ub.shape[1])

res = linprog(c=c_obj, A_ub=A_ub, b_ub=b_ub, options={"disp": False, "maxiter":10})
res

     con: array([], dtype=float64)
     fun: 0.4977413120628635
 message: 'The algorithm terminated successfully and determined that the problem is infeasible.'
     nit: 4
   slack: array([-1.05919958, -0.84646709, -1.3662654 , -1.57899789])
  status: 2
 success: False
       x: array([0.23784216, 0.25989916])

#### The data is separable by labels, so the LP should have produced an optimal solution. Any values between -1 to 1 is a solution which is separating vector 

# Scipy Way (Separable) Shafi Version

In [9]:
t = y.values
float32_epsilon = (np.finfo(np.float32).eps)*10
xx = np.array(t.reshape(-1,1) * X.values)
neg_ones = np.repeat(-1, X.shape[0]).reshape(-1,1)

A_ub = np.array(neg_ones * xx)

b_ub = np.repeat(-1*float32_epsilon, X.shape[0]).reshape(-1,1)

c_obj = np.repeat(0, X.shape[1])

res = linprog(c=c_obj, A_ub=A_ub, b_ub=b_ub, options={"disp": False, "maxiter":10})
res

ValueError: The problem is (trivially) infeasible because a singleton row in the upper bound constraints is inconsistent with the bounds.

#### In this case a solution is reached

# Gurobi Way (Separable)

In [17]:
X_separable = np.array([[1],[7],[8],[9]])
y_separable = np.array([[-1],[-1],[1],[1]])
X = pd.DataFrame(X_separable)
n,p = X.shape
y = np.squeeze(y_separable)
m = Model()
v = []


# Define variables and add to objective function
for i in range(p):
    v+= [m.addVar(-GRB.INFINITY,GRB.INFINITY,0,GRB.CONTINUOUS,"v"+str(i))]

v+= [m.addVar(-GRB.INFINITY,GRB.INFINITY,0,GRB.CONTINUOUS,"c")]
m.update()
X['ones'] = np.ones(n)

float32_epsilon = (np.finfo(np.float32).eps)*10

# Constraints
for i in range(n):
    if y[i]>0:
        m.addConstr(((X.iloc[i]).dot(v))>= 1)
    else:
        m.addConstr(((X.iloc[i]).dot(v))<= 0)

m.update()

# m.ModelSense = -1
# m.Params.OutputFlag = 0 # To avoid verbose output of m.optimize()
m.optimize() # Run the model

if m.status == 3:
    model_status = 'Infeasible'
else:
    model_status = 'Optimal or Others'
print('Model Status:',model_status)

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 4 rows, 2 columns and 8 nonzeros
Model fingerprint: 0xa7e41267
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.150714e-01   0.000000e+00      0s
       1    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds
Optimal objective  0.000000000e+00
Model Status: Optimal or Others


In [18]:
# A solution is reached, below is the solution

In [19]:
for v in m.getVars():
        print('%s %s %g' % (v.varName,':', v.x))

v0 : 1
c : -7


# Inseparable

In [26]:
X = pd.DataFrame(X_nonseparable)
X

,0
0,-2
1,-1
2,1
3,2


In [27]:
y = pd.DataFrame(y_nonseparable)
y

,0
0,-1
1,1
2,-1
3,1


# Scipy Way (Inseparable)

In [29]:
tmp = X.values
tmp = sc.fit_transform(tmp)


xx = np.array(y.values.reshape(-1,1) * tmp)
t = y.values

A_ub = np.append(xx, t.reshape(-1,1), 1)
b_ub = np.repeat(-1, A_ub.shape[0]).reshape(-1,1)

c_obj = np.repeat(1, A_ub.shape[1])

res = linprog(c=c_obj, A_ub=A_ub, b_ub=b_ub, options={"disp": False, "maxiter":10})
res

     con: array([], dtype=float64)
     fun: 3.3294229710976895
 message: 'The algorithm terminated successfully and determined that the problem is infeasible.'
     nit: 4
   slack: array([-1.91161049, -1.272659  ,  1.64119799, -4.82546748])
  status: 2
 success: False
       x: array([1.87249448, 1.45692849])

# Gurobi Way (Inseparable)

In [164]:
X_nonseparable = np.array([[-2],[-1],[1],[2]])
y_nonseparable = np.array([[-1],[1],[-1],[1]])
X = pd.DataFrame(X_nonseparable)
n,p = X.shape
y = np.squeeze(y_nonseparable)
m = Model()
v = []


# Define variables and add to objective function
for i in range(p):
    v+= [m.addVar(-GRB.INFINITY,GRB.INFINITY,0,GRB.CONTINUOUS,"v"+str(i))]

v+= [m.addVar(-GRB.INFINITY,GRB.INFINITY,0,GRB.CONTINUOUS,"c")]
m.update()
X['ones'] = np.ones(n)

float32_epsilon = (np.finfo(np.float32).eps)*10

# Constraints
for i in range(n):
    if y[i]>0:
        m.addConstr(y[i]*((X.iloc[i]).dot(v))>= 0)
    else:
        m.addConstr(y[i]*((X.iloc[i]).dot(v))>= float32_epsilon)

m.update()

# m.ModelSense = -1
# m.Params.OutputFlag = 0 # To avoid verbose output of m.optimize()
m.optimize() # Run the model

if m.status == 3:
    model_status = 'Infeasible'
else:
    model_status = 'Optimal or Others'
print('Model Status:',model_status)

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 4 rows, 2 columns and 8 nonzeros
Model fingerprint: 0x4c1c562d
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-06, 1e-06]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.384186e-06   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Infeasible model
Model Status: Infeasible


# Break Down of Codes in Scipy way

In [39]:
X = pd.DataFrame(X_separable)
X

,0,1,2
0,1,1,1
1,1,2,3
2,1,2,5
3,1,5,5


In [40]:
y= pd.DataFrame(y_separable)
y

,0
0,-1
1,-1
2,1
3,1


In [41]:
tmp = X.values
tmp

array([[1, 1, 1],
       [1, 2, 3],
       [1, 2, 5],
       [1, 5, 5]])

In [42]:
tmp = sc.fit_transform(tmp)
tmp

array([[ 0.        , -1.        , -1.50755672],
       [ 0.        , -0.33333333, -0.30151134],
       [ 0.        , -0.33333333,  0.90453403],
       [ 0.        ,  1.66666667,  0.90453403]])

In [43]:
xx = np.array(y.values.reshape(-1,1) * tmp)
xx

array([[-0.        ,  1.        ,  1.50755672],
       [-0.        ,  0.33333333,  0.30151134],
       [ 0.        , -0.33333333,  0.90453403],
       [ 0.        ,  1.66666667,  0.90453403]])

In [44]:
t = y.values
t

array([[-1],
       [-1],
       [ 1],
       [ 1]])

In [46]:
A_ub = np.append(xx, t.reshape(-1,1), 1)
A_ub

array([[-0.        ,  1.        ,  1.50755672, -1.        ],
       [-0.        ,  0.33333333,  0.30151134, -1.        ],
       [ 0.        , -0.33333333,  0.90453403,  1.        ],
       [ 0.        ,  1.66666667,  0.90453403,  1.        ]])

In [47]:
b_ub = np.repeat(-1, A_ub.shape[0]).reshape(-1,1)
b_ub

array([[-1],
       [-1],
       [-1],
       [-1]])

In [48]:
c_obj = np.repeat(1, A_ub.shape[1])
c_obj

array([1, 1, 1, 1])

In [49]:
res = linprog(c=c_obj, A_ub=A_ub, b_ub=b_ub, options={"disp": False, "maxiter":1000})
res

     con: array([], dtype=float64)
     fun: 1.2790274871453626
 message: 'The algorithm terminated successfully and determined that the problem is infeasible.'
     nit: 4
   slack: array([-1.97569017, -1.12061871, -1.21656849, -2.7702213 ])
  status: 2
 success: False
       x: array([0.        , 0.7768264 , 0.27958086, 0.22262023])

## Let's try another example

In [160]:
X_separable = np.array([[1,1],[2,3],[2,5],[5,5]])
y_separable = np.array([[-1],[-1],[1],[1]])

# X_separable = np.array([[1,1],[2,2],[3,3],[4,4]])
# y_separable = np.array([[-1],[-1],[1],[1]])

# X_separable = np.array([[-2],[-1],[1],[2]])
# y_separable = np.array([[-1],[-1],[1],[1]])

In [161]:
X = pd.DataFrame(X_separable)
n,p = X.shape
y = np.squeeze(y_separable)
m = Model()
v = []


# Define variables and add to objective function
for i in range(p):
    v+= [m.addVar(-GRB.INFINITY,GRB.INFINITY,0,GRB.CONTINUOUS,"v"+str(i))]

v+= [m.addVar(-GRB.INFINITY,GRB.INFINITY,0,GRB.CONTINUOUS,"c")]
m.update()
X['ones'] = np.ones(n)

float32_epsilon = (np.finfo(np.float32).eps)*10

# Constraints
for i in range(n):
    if y[i]>0:
        print('yaya')
        m.addConstr(y[i]*((X.iloc[i]).dot(v))>= 0)
    else:
        print('jaja')
        m.addConstr(y[i]*((X.iloc[i]).dot(v))>= float32_epsilon)

m.update()

# m.ModelSense = -1
# m.Params.OutputFlag = 0 # To avoid verbose output of m.optimize()
m.optimize() # Run the model

if m.status == 3:
    model_status = 'Infeasible'
else:
    model_status = 'Optimal or Others'
print('Model Status:',model_status)

jaja
jaja
yaya
yaya
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 4 rows, 3 columns and 12 nonzeros
Model fingerprint: 0x6f54215c
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-06, 1e-06]
Presolve removed 4 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  0.000000000e+00
Model Status: Optimal or Others


In [162]:
for v in m.getVars():
        print('%s %s %g' % (v.varName,':', v.x))

v0 : 0
v1 : 5.96046e-07
c : -2.98023e-06


In [21]:
t = y
float32_epsilon = (np.finfo(np.float32).eps)*10
xx = np.array(t.reshape(-1,1) * X.values)
neg_ones = np.repeat(-1, X.shape[0]).reshape(-1,1)

A_ub = np.array(neg_ones * xx)

b_ub = np.repeat(-1*float32_epsilon, X.shape[0]).reshape(-1,1)

c_obj = np.repeat(0, X.shape[1])

res = linprog(c=c_obj, A_ub=A_ub, b_ub=b_ub, options={"disp": False, "maxiter":10})
res

     con: array([], dtype=float64)
     fun: 0.0
 message: 'The algorithm terminated successfully and determined that the problem is infeasible.'
     nit: 5
   slack: array([-0.00046196, -0.00092273,  0.00138112,  0.00184189])
  status: 2
 success: False
       x: array([0.00023039, 0.00023039])